# Vanilla CNN
---

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
import numpy as np

## Hyperparameters

In [2]:
EPOCHS = 10

## Model Definition

In [3]:
class ConvNet(tf.keras.Model):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.sequence = list()
        # 'Conv2D()': API for CNN
        # the first arg: the number of output channels
        # API recognizes the number of input channels itself.
        # (no need to insert the number of input channels manually)
        # the second arg: the size of kernel
        # 'padding': default -> 'valid' (no zero padding)
        #            'same' -> zero padding
        # 28(H) X 28(W) X 16(C)
        self.sequence.append(Conv2D(16, (3, 3), padding = 'same',
                                    activation = 'relu'))
        # 28(H) X 28(W) X 16(C)
        self.sequence.append(Conv2D(16, (3, 3), padding = 'same',
                                    activation = 'relu'))
        # the arg of MaxPool2D: the size of pooling
        # 14(H) X 14(W) X 16(C)
        self.sequence.append(MaxPool2D((2, 2)))
        # 14(H) X 14(W) X 32(C)
        self.sequence.append(Conv2D(32, (3, 3), padding = 'same',
                                    activation = 'relu'))
        # 14(H) X 14(W) X 32(C)
        self.sequence.append(Conv2D(32, (3, 3), padding = 'same',
                                    activation = 'relu'))
        # 7(H) X 7(W) X 32(C)
        self.sequence.append(MaxPool2D((2, 2)))
        # 7(H) X 7(W) X 64(C)
        self.sequence.append(Conv2D(64, (3, 3), padding = 'same',
                                    activation = 'relu'))
        # 7(H) X 7(W) X 64(C)
        self.sequence.append(Conv2D(64, (3, 3), padding = 'same',
                                    activation = 'relu'))
        # the size of flattened vector -> 3136 (7 X 7 X 64)
        self.sequence.append(Flatten())
        self.sequence.append(Dense(128, activation = 'relu'))
        self.sequence.append(Dense(10, activation = 'softmax'))
        
    def call(self, x, training = False, mask = None):
        for layer in self.sequence:
            x = layer(x)
        return x

## Training and Test Loops

In [4]:
# Implementing Training Loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implementing Test Loop
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)
    
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## Dataset

In [5]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# rescaling
x_train, x_test = x_train / 255.0, x_test / 255.0 # x_train and x_test
                                                  # become float64.
# '..., tf.newaxis': adding an axis (axis for channel)
# N X 28 X 28 -> N X 28 X 28 X 1 (N: the sample size of MNIST)
# change the types of x_train and x_test into float32
x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)
# 'from_tensor_slices': enables users to construct a dataset for a network
# when they insert numpy objects or tensor objects.
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Training Environment

In [6]:
# Creating a model
model = ConvNet()

# Loss Function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

# Optimizer (algorithm for optimization)
optimizer = tf.keras.optimizers.Adam()

# evaluation metric
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')
test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

## Training

In [8]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer,
                  train_loss, train_accuracy)
        
    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object,
                  test_loss, test_accuracy)
    
    print(f'Epoch: {epoch + 1}, ' +
         f'Loss: {train_loss.result()}, ' +
         f'Accuracy: {train_accuracy.result() * 100}, ' +
         f'Test Loss: {test_loss.result()}, ' +
         f'Test Accuracy: {test_accuracy.result() * 100}')

Epoch: 1, Loss: 0.08112213760614395, Accuracy: 97.52583312988281, Test Loss: 0.03843991085886955, Test Accuracy: 98.7699966430664
Epoch: 2, Loss: 0.06439409404993057, Accuracy: 98.01499938964844, Test Loss: 0.034841831773519516, Test Accuracy: 98.92500305175781
Epoch: 3, Loss: 0.0543186254799366, Accuracy: 98.32500457763672, Test Loss: 0.03318794444203377, Test Accuracy: 98.9366683959961
Epoch: 4, Loss: 0.04779471829533577, Accuracy: 98.52633666992188, Test Loss: 0.0336960069835186, Test Accuracy: 98.93500518798828
Epoch: 5, Loss: 0.0426211841404438, Accuracy: 98.68582916259766, Test Loss: 0.0330728217959404, Test Accuracy: 98.96600341796875
Epoch: 6, Loss: 0.03863511607050896, Accuracy: 98.80738067626953, Test Loss: 0.03452499955892563, Test Accuracy: 98.96333312988281
Epoch: 7, Loss: 0.0357552170753479, Accuracy: 98.89500427246094, Test Loss: 0.0335228405892849, Test Accuracy: 99.00143432617188
Epoch: 8, Loss: 0.03304910659790039, Accuracy: 98.977783203125, Test Loss: 0.0356005802750